counter = {}

def subst(x, y):
  return 3 if x == y else -1

def align(s1, s2, g = -4):
  key = f"{s1}:{s2}"
  counter[key] = counter.get(key, 0) + 1
  if not s1 or not s2:
    return max(len(s1), len(s2)) * g

  return max(
      align(s1[:-1], s2[:-1]) + subst(s1[-1], s2[-1]),
      align(s1[:-1], s2) + g,
      align(s1, s2[:-1]) + g)
  
align('A' * 10, 'B' * 10)
counter

max(counter.values())

tab = {}
def fib(n):
  if n < 2:
    return 1
  if n in tab: return tab[n]
  tab[n] =  fib(n - 1) + fib(n - 2)
  return tab[n]

fib(100)

In [14]:

import io

class Mat:
    """Implementsção de uma matris
    """
    def __init__(self, rows, cols):
        #print(f"construtor do Mat invocado com {rows} e {cols}")
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        "Devolve a matriz como uma string"
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        "Utilizado no repl quando se pede para ver a matriz"
        return str(self)
    def __getitem__ (self, n):
        "Interface para a indexação []"
        return self.mat[n]

"""
2 chars iguais -> +3
diferentes     -> -1
"""

def subst(x, y):
  import blosum as bl
  dic = dict(bl.BLOSUM(62))
  return int(dic[x+y])   # feito 
                     
class NW:
  """
  Alinhamento global entre duas sequências
  """
  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)  # este + 1 por causa dos "cabeçalhos" com as seqs
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):                     # prencher a primeira linha
      self.mat[L + 1][0] = g * (L + 1)            # na matriz 
      self.tr[L + 1][0]  = 'C'                      # no Trace

    for C in range(len(s2)):                     # prencher a primeira coluna
      self.mat[0][C + 1] = g * (C + 1)             #na matriz
      self.tr[0][C + 1]  = 'E'                        # no trace

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)   #colocar na matriz o valor max das 3 possibilidades
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]  # colocar D E C dependendo da posiçao onde foi obtido o max (self,mat[L +1][C + 1])

  def rebuild(self):
    # Começar no canto inferior direito
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0)
    }

    while L > 0 or C > 0:
      #print(self.s1[L - 1], self.s2[C - 1])
      #print(L, C)
      #print(self.tr[L][C])
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1    # este self.s1/s2 representa as sequencias 
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        
      #print(S1, S2)
      L += DL
      C += DC

    return S1, S2

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()
    
class SW(NW):
  """
  Alinhamento local entre duas sequências
  """
  def __init__(self, s1, s2, g = -4):
      self.s1 = s1
      self.s2 = s2
      self.mat = Mat(len(s1) + 1, len(s2) + 1)

      for L, x1 in enumerate(s1):
        for C, x2 in enumerate(s2):
          self.mat[L + 1][C + 1] = max(
              self.mat[L  ][C    ] + subst(x1, x2),
              self.mat[L+1][C    ] + g,
              self.mat[L  ][C + 1] + g,
              0
          )

alin = NW("ACGT", "TACCGT")
print(alin)

#TACCGT
#-A-CGT

print(*alin.rebuild(), sep="\n")

    -   T   A   C   C   G   T
-   0  -4  -8 -12 -16 -20 -24
A  -4   0   0  -4  -8 -12 -16
C  -8  -4   0   9   5   1  -3
G -12  -8  -4   5   6  11   7
T -16  -7  -8   1   4   7  16

  - T A C C G T
- 0 E E E E E E
A C D D E E E E
C C C D D D E E
G C C D C D D E
T C D D C D C D

-A-CGT
TACCGT
